# Polynomial Regression - Elastic Net

### Import Libraries

In [1]:
import pandas as pd
import numpy  as np

from sklearn import metrics       as mt
from sklearn import preprocessing as pp
from sklearn import linear_model  as lm

### Load Dataset

In [2]:
X_train = pd.read_csv('Training/X_training.csv')
y_train = pd.read_csv('Training/y_training.csv')
X_val = pd.read_csv('Validation/X_val.csv')
y_val = pd.read_csv('Validation/y_val.csv')
X_test = pd.read_csv('Test/X_test.csv')
y_test = pd.read_csv('Test/y_test.csv')

In [3]:
d = np.arange( 1, 4)
a = np.arange( 1, 5)
#l1_ratios = np.arange(0.1, 1.0, 0.1)
rmse_list = []
max_r2 = 0
min_mse = float('inf')
min_rmse = float('inf')
min_mae = float('inf')
min_mape = float('inf')

for i in d:
    for alpha in a:
        
        # Define
        poly = pp.PolynomialFeatures(degree=i)
        X_poly_train = poly.fit_transform(X_train)

        # Training
        model = lm.ElasticNet(alpha=alpha, max_iter=1000)
        model.fit(X_poly_train, y_train)

        # Predict
        yhat_train = model.predict(X_poly_train)

        r2 = mt.r2_score(y_train, yhat_train)
        if r2 > max_r2:
            max_r2 = r2    

        mse = mt.mean_squared_error(y_train, yhat_train)
        if mse < min_mse:
            min_mse = mse    

        rmse = np.sqrt(mse)
        rmse_list.append(rmse)
        if rmse < min_rmse:
            min_rmse = rmse    

        mae = mt.mean_absolute_error(y_train, yhat_train)
        if mae < min_mae:
            min_mae = mae    

        mape = mt.mean_absolute_percentage_error(y_train, yhat_train)
        if mape < min_mape:
            min_mape = mape

print(f'Degree: {i}')
print(f'Alpha: {alpha}')
print(f'Max R2: {max_r2:.3f}')
print(f'Min MSE: {min_mse:.3f}')
print(f'Min RMSE: {min_rmse:.3f}')
print(f'Min MAE: {min_mae:.3f}')
print(f'Min MAPE: {min_mape:.3f}')    

Degree: 3
Alpha: 4
Max R2: 0.018
Min MSE: 469.323
Min RMSE: 21.664
Min MAE: 17.187
Min MAPE: 8.607


### Validation

In [7]:
d = np.arange( 1, 4)
a = np.arange( 1, 5)
l1_ratios = np.arange(0.1, 1.0, 0.1)
rmse_list = []
max_r2 = 0
min_mse = float('inf')
min_rmse = float('inf')
min_mae = float('inf')
min_mape = float('inf')
best_degree = 0
best_alpha = 0
best_l1_ratio = 0

for i in d:
    for alpha in a:
      for l1 in l1_ratios:
         poly = pp.PolynomialFeatures(degree=i)
         X_poly_train = poly.fit_transform(X_train)
         X_poly_val = poly.fit_transform(X_val)

         # Training
         model = lm.ElasticNet(alpha=alpha, l1_ratio=l1, max_iter=1000)
         model.fit(X_poly_train, y_train)

         # Predict
         yhat_val = model.predict(X_poly_val)

         r2 = mt.r2_score(y_val, yhat_val)
         if r2 > max_r2:
            max_r2 = r2
            best_degree = i
            best_alpha = alpha
            best_l1_ratio = l1

         mse = mt.mean_squared_error(y_val, yhat_val)
         if mse < min_mse:
            min_mse = mse   

         rmse = np.sqrt(mse)
         rmse_list.append( rmse )
         if rmse < min_rmse:
            min_rmse = rmse

         mae = mt.mean_absolute_error(y_val, yhat_val)
         if mae < min_mae:
            min_mae = mae 

         mape = mt.mean_absolute_percentage_error(y_val, yhat_val)
         if mape < min_mape:
            min_mape = mape

print(f'The best Degreee: {best_degree}\n'
      f'The best Alpha: {best_alpha}\n'
      f'The best L1 Ratio: {best_l1_ratio}\n'
      f'R2: {max_r2:.3f}\n'
      f'MSE: {min_mse:.3f}\n'
      f'RMSE: {min_rmse:.3f}\n'
      f'MAE: {min_mae:.3f}\n'
      f'MAPE: {min_mape:.3f}\n'
    )

Degreee: 3
Alpha: 19
R2: 0.014
MSE: 470.756
RMSE: 21.697
MAE: 17.181
MAPE: 8.656



### Test

In [8]:
# The best parameters retraining with test data

# Define
poly = pp.PolynomialFeatures(degree=best_degree)
X_poly_train = poly.fit_transform(X_train)
X_poly_val = poly.fit_transform(X_val)
X_poly_test = poly.fit_transform(X_test)

# Training
model = lm.ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio, max_iter=1000)
model.fit( np.concatenate( ( X_poly_train, X_poly_val ) ),
           np.concatenate( ( y_train, y_val ) ) )

# Predict
y_pred = model.predict(X_poly_test)

r2 = mt.r2_score(y_test, y_pred)

mse = mt.mean_squared_error(y_test, y_pred)   

rmse = np.sqrt(mse)

mae = mt.mean_absolute_error(y_test, y_pred)

mape = mt.mean_absolute_percentage_error(y_test, y_pred)

print(f'The best Degreee: {best_degree}\n'
      f'Alpha: {best_alpha}\n'
      f'L1 Ratio: {best_l1_ratio}\n'
      f'R2: {r2:.3f}\n'
      f'MSE: {mse:.3f}\n'
      f'RMSE: {rmse:.3f}\n'
      f'MAE: {mae:.3f}\n'
      f'MAPE: {mape:.3f}\n'
    )

Degreee: 3
Alpha: 19
R2: 0.009
MSE: 482.615
RMSE: 21.968
MAE: 17.441
MAPE: 8.714

